In [ ]:
import numpy as np
import pandas as pd

# Data Importing + Cleaning

In [ ]:
train_df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
train_df.head(5)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def clean_df(df):
    ohe = OneHotEncoder(sparse=False)
    columns_to_remove = ["PassengerId", "Name", "Cabin"]
    df = df.drop(columns=columns_to_remove)
    categorical_features = ["HomePlanet", "Destination"]
    encoded_features = ohe.fit_transform(df[categorical_features])
    encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(categorical_features))
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(categorical_features, axis=1, inplace=True)
    df["const"] = 1
    df = df.fillna(0).astype(int)
    return df

In [ ]:
encoded_train_df = clean_df(train_df)
encoded_train_df.info()
x = np.array(encoded_train_df.drop(columns=["Transported"]))
y = np.array(encoded_train_df["Transported"])

In [ ]:
x

In [ ]:
y.shape

# Attempts To Create the model

## Attempt 1

Based off of: https://www.kaggle.com/code/wwsalmon/simple-mnist-nn-from-scratch-numpy-no-tf-keras

In [ ]:
def initialize_weights():
    w = np.random.rand(10, 17)
    w2 = np.random.rand(2, 10)
    return w, w2

def relu(z):
    return np.maximum(z, 0)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def propagation(w, w2, x):
    z1 = w.dot(x)
    a1 = relu(z1)
    z2 = w2.dot(a1)
    a2 = sigmoid(z2)
    return z1, a1, z2, a2
    
def dir_relu(z):
    return z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(z1, a1, z2, a2, w1, w2, x, y):
    one_hot_y = one_hot(y)
    m = y.size
    dz2 = a2 - one_hot_y
    dw2 = 1/m * dz2.dot(a1.T)
    dz1 = w2.T.dot(dz2) * dir_relu(z1)
    dw1 = 1/m * dz1.dot(x.T)
    return dw1, dw2

def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

def updated_weights(w1, w2, dw1, dw2, lr):
    w1 = w1 - (lr * dw1)
    w2 = w2 - (lr * dw2)
    return w1, w2

def gd(x, y, lr, epochs):
    w1, w2 = initialize_weights()
    for i in range(epochs):
        z1, a1, z2, a2 = propagation(w1, w2, x)
        dw1, dw2 = back_prop(z1, a1, z2, a2, w1, w2, x, y)
        w1, w2 = updated_weights(w1, w2, dw1, dw2, lr)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a2)
            print(get_accuracy(predictions, y))
    return w1, w2

In [ ]:
w1, w2 = gd(x.T, y.T, .01, 100)

I couldn't get this one to work, so instead of continuing to spin my wheels on it I decided to scrap it and start completely over.

## Attempt 2

In [ ]:
def initialize_weights():
    weights = np.random.rand(17)
    return weights

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def get_predictions(a):
    return np.round(a)

def get_accuracy(predictions, y):
    return np.sum(predictions == y) / y.size

def gd(x, y, lr, epochs):
    w = initialize_weights()
    
    for i in range(epochs):
        z = np.dot(x, w)
        a = sigmoid(z)
        dw = (1/x.shape[0]) * np.dot(x.T, (a-y))
        w -= lr * dw
        
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a)
            print(get_accuracy(predictions, y))
    return w

def predict(x, w):
    z = np.dot(x, w)
    a = sigmoid(z)
    predictions = get_predictions(a)
    return predictions

In [ ]:
# I found using 40 epochs gave the best results without overfitting
# w1 = gd(x, y, .01, 40)

In [ ]:
# cleaned_test_df = clean_df(test_df)
# test_x = np.array(cleaned_test_df)
# predictions = predict(test_x, w1)

In [ ]:
# test_df["Transported"] = predictions
# test_df["Transported"] = test_df["Transported"].astype(bool)
# test_df[["PassengerId", "Transported"]].to_csv("submission.csv", index=False)

## Attempt 3

In [ ]:
def initialize_weights():
    w1 = np.random.randn(30, 17)
    w2 = np.random.randn(10, 30)
    w3 = np.random.randn(1, 10)
    return w1, w2, w3

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def get_predictions(a):
    return np.round(a)

def get_accuracy(predictions, y):
    return np.sum(predictions == y) / y.size

def forward_prop(x, w1, w2, w3):
    z = np.dot(w1, x)
    a1 = sigmoid(z)
    z2 = np.dot(w2, a1)
    a2 = sigmoid(z2)
    z3 = np.dot(w3, a2)
    a3 = sigmoid(z3)
    return a1, a2, a3


def back_prop(x, y, a1, a2, a3, w1, w2, w3):
    batch_size = x.shape[1]  # Get batch size
    dz3 = a3 - y
    d_w3 = (1/batch_size) * np.dot(dz3, a2.T)
    dz2 = np.dot(w3.T, dz3) * a2 * (1 - a2)
    d_w2 = (1 / batch_size) * np.dot(dz2, a1.T)
    dz1 = np.dot(w2.T, dz2) * a1 * (1 - a1)
    d_w1 = (1 / batch_size) * np.dot(dz1, x.T)
    
    return d_w1, d_w2, d_w3

def gd(x, y, lr, epochs):
    w1, w2, w3 = initialize_weights()
    
    for i in range(epochs):
        a1, a2, a3 = forward_prop(x, w1, w2, w3)
        dw1, dw2, dw3 = back_prop(x, y, a1, a2, a3, w1, w2, w3)

        w1 -= lr * dw1
        w2 -= lr * dw2
        w3 -= lr * dw3
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(a3)
            print(get_accuracy(predictions, y))
    return w1, w2, w3

def predict(x, w1, w2, w3):
    _, _, a3 = forward_prop(x.T, w1, w2, w3)
    predictions = get_predictions(a3)
    return predictions


In [ ]:
w1, w2, w3 = gd(x.T, y, .07, 500)

In [ ]:
cleaned_test_df = clean_df(test_df)
test_x = np.array(cleaned_test_df)
predictions = predict(test_x, w1, w2, w3)
test_df["Transported"] = predictions.T
test_df["Transported"] = test_df["Transported"].astype(bool)
test_df[["PassengerId", "Transported"]].to_csv("submission.csv", index=False)